In [1]:
import numpy as np
import pandas as pd
import os
from collections import namedtuple


In [2]:
from PIL import Image, ImageDraw

In [3]:
%cd C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection

C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection


In [4]:
import torch
import torchvision
from torch import Tensor
from torchvision import transforms as T
from torch.utils.data import DataLoader

In [5]:
DatatransformationConfig = namedtuple("data_transformation", [
  'root_dir',
  'local_data_file',
  'transformed_data_file'
])

In [6]:
from faster_rcnn_od.constants import *
from faster_rcnn_od.utils.comman import read_yaml, create_directories, custom_collate, CustDat
from tqdm.auto import tqdm

c:\Users\anil\anaconda3\envs\fasterrcnn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transform_config(self) -> DatatransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DatatransformationConfig(
            root_dir=config.root_dir,
            local_data_file=config.local_data_file,
            transformed_data_file=config.transformed_data_file 
        )

        return data_transformation_config

In [21]:

class DataTransform_load:
    def __init__(self, config: DatatransformationConfig ):
        self.config = config

    
    def transform_file(self):
        
        if  os.path.exists(self.config.local_data_file):
            self.data = pd.read_csv(self.config.local_data_file)
            self.df = self.data[['filename','xmin', 'ymin', 'xmax','ymax', 'class']]
            self.df['class'] = self.df['class'].map({'Sickle': 1, 'Normal': 2,'Target': 3, 'Other': 4, 'Crystal': 5 })
            self.unique_imgs = self.df['filename'].unique()
            from sklearn.model_selection import train_test_split
            self.train_inds, self.val_inds = train_test_split(range(self.unique_imgs.shape[0]), test_size = 0.2)
    
            #self.save_file(self.config.transformed_data_file, self.df)
            return self.df, self.unique_imgs, self.train_inds, self.val_inds
            #return self.unique_imgs
            
    def get_cust_data(self):
        df, img, train_idx, val_idx = self.transform_file()
        self.train_dl = DataLoader(CustDat(df, img, train_idx),
                              batch_size=1,
                              shuffle=True,
                              collate_fn = custom_collate,
                              pin_memory = False)
        
        self.val_dl = DataLoader(CustDat(df, img, val_idx),
                              batch_size=1,
                              shuffle=True,
                              collate_fn = custom_collate,
                              pin_memory = False)
        self.save_file(self.config.transformed_data_file, self.train_dl, 'train_loder')
        self.save_file(self.config.transformed_data_file, self.val_dl, 'val_loader')

            
    

    def save_file(self, path, train_dl, file_name):
        print(file_name)
        path_trn = os.path.join(path, file_name)
        torch.save(train_dl, path_trn+".pt")
        


In [22]:
config = ConfigurationManager()
prepare_data_config = config.get_data_transform_config()
prepare_base = DataTransform_load(config=prepare_data_config)
prepare_base.get_cust_data()


[2023-05-06 17:01:49,001: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-05-06 17:01:49,006: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-05-06 17:01:49,008: INFO: comman: created directory at: artifacts]
[2023-05-06 17:01:49,010: INFO: comman: created directory at: artifacts/transformed_data]
train_loder
val_loader


C:\Users\anil\AppData\Local\Temp\ipykernel_11012\591461073.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['class'] = self.df['class'].map({'Sickle': 1, 'Normal': 2,'Target': 3, 'Other': 4, 'Crystal': 5 })
